# RAG-SQL

# Installing Dependencies

In [1]:
!pip install langchain langchain-experimental openai pymysql

   ---------------------------------------- 0.0/206.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/206.9 kB ? eta -:--:--
   ----- --------------------------------- 30.7/206.9 kB 435.7 kB/s eta 0:00:01
   ------------------ ------------------- 102.4/206.9 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 206.9/206.9 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 45.0/45.0 kB 2.2 MB/s eta 0:00:00


# Importing necessary Packages

In [25]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage
from langchain.prompts.chat import HumanMessagePromptTemplate

# Initialize the LLM

In [3]:
OPEN_API_KEY="Api_key"
llm=ChatOpenAI(temperature=0,openai_api_key=OPEN_API_KEY)

C:\Users\sravanthi\AppData\Local\Temp\ipykernel_21816\576305831.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm=ChatOpenAI(temperature=0,openai_api_key=OPEN_API_KEY)


# DataBase Setup

In [5]:
# Database connection details such as host, port, username, password, and schema are set for MySQL connection
host='localhost'
port='3306'
username='root'
password='12345'
database_schema='movies'

# Forming the MySQL URI string for connecting to the database using pymysql
mysql_uri= f"mysql+pymysql://{username}:{password}@{host}:{port}/movies"

# Creating an SQLDatabase object from the given URI and including the 'telugumovies_dataset' table with sample rows for metadata
db=SQLDatabase.from_uri(mysql_uri,include_tables=['telugumovies_dataset'],sample_rows_in_table_info=2)

# Creating an SQLDatabaseChain object with the specified language model (llm) and database connection (db), with verbose mode enabled for detailed output
db_chain=SQLDatabaseChain.from_llm(llm,db,verbose=True)


In [7]:
# Function to execute a query on the SQLDatabaseChain and return the result as a string
def retrieve_from_db(query: str) -> str:
    # Executing the query using the db_chain object
    db_context = db_chain(query)
    
    # Extracting and trimming the 'result' from the returned context
    db_context = db_context['result'].strip()
    
    # Returning the cleaned result as a string
    return db_context


# Returning the Results

In [27]:
# Function to generate a response for a user's query related to Telugu movies using the database and an LLM
def generate(query: str) -> str:
    # Retrieve the relevant data from the database based on the query
    db_context = retrieve_from_db(query)
    
    # Define the system message that sets the context for the LLM to act as a Telugu movie database expert
    system_message = """
    You are a movie database expert specializing in Telugu cinema. 
    Your task is to answer users' questions by providing relevant information from the database of Telugu movies.
    The database contains the following information for each movie:
      - Name of the movie
      - Year of release
      - Certificate given by the censor board
      - Movie genre
      - A brief description or plot of the movie
      - Duration in minutes
      - IMDb rating
      - Number of people who rated the movie

    Example:
    Input:
    Which Telugu movie released in 2021 has the highest IMDb ratings?
    
    Context:
    The Telugu movies released in 2021 with the highest IMDb ratings are:
    1. Movie A - IMDb rating: 8.5
    2. Movie B - IMDb rating: 8.3
    
    Output:
    The highest-rated Telugu movies released in 2021 are Movie A (IMDb rating: 8.5) and Movie B (IMDb rating: 8.3).
    """
    
    # Creating a template for the human message prompt, combining user input and database context
    human_qry_template = HumanMessagePromptTemplate.from_template("""
    Input: {human_input}
    Context: {db_context}
    Output:
    """)
    
    # Preparing the messages for the LLM with the system message and the formatted human query template
    messages = [
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query, db_context=db_context)
    ]
    
    # Generate the response using the LLM with the provided messages
    response = llm(messages).content
    
    # Return the generated response
    return response


# Generate and Return the responses

In [29]:
generate("How many movies are there?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


How many movies are there?
SQLQuery:SELECT COUNT(`Movie`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1350,)]
Answer:There are 1350 movies in the dataset.
> Finished chain.


'There are 1350 movies in the dataset.'

In [31]:
generate("which movie have highest rating in 2021?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


which movie have highest rating in 2021?
SQLQuery:SELECT `Movie`, `Rating`
FROM telugumovies_dataset
WHERE `Year` = 2021
ORDER BY `Rating` DESC
LIMIT 1;
SQLResult: 
Answer:Baahubali 2: The Conclusion
> Finished chain.


"I'm sorry, but I need more information to provide an accurate answer. Can you please specify if you are asking about the highest-rated movie in general or the highest-rated Telugu movie released in 2021?"

In [33]:
generate("According to IMDb rating suggest the best movie to watch?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


According to IMDb rating suggest the best movie to watch?
SQLQuery:SELECT `Movie`, `Rating` 
FROM telugumovies_dataset 
ORDER BY `Rating` DESC 
LIMIT 1;
SQLResult: [('Pichhodu', 9.5)]
Answer:Pichhodu
> Finished chain.


"I'm sorry, but I need more information to provide a recommendation based on IMDb ratings. Could you please provide me with a list of Telugu movies along with their IMDb ratings?"